#### Code adapted from https://github.com/philippe-heitzmann/NYSDA_WSJ_Web_Scraping_Project2/blob/master/wsj_scraping_test.py , and modified for a data collection scheme that saves the text data into a Microsoft Excel spreadsheet

In [ ]:
#Packages needed
from selenium import webdriver
import time
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup 
import csv
import os.path

In [ ]:
# THE CREDIT GOES TO HEITZMANN (REFERENCE IN THE FIRST CELL) FOR ALL OF THE CODE IN THIS CELL

# Path to chromedriver location
PATH = 'YOUR PATH TO CHROMEDRIVER HERE'
driver = webdriver.Chrome(PATH)

url = 'https://www.wsj.com/news/archive/"PLACE YEAR HERE"/"PLACE MONTH HERE"'
driver.get(url)

sign_in_link = driver.find_element_by_link_text('Sign In')
sign_in_link.click()

username = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, 'username')))
password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, 'password')))

user1 = 'YOUR USERNAME'
pass1 = 'YOUR PASSWORD'

username.send_keys(user1)
password.send_keys(pass1)

submit_button = driver.find_element_by_xpath('//*[@id="basic-login"]/div[1]/form/div/div[6]/div[1]/button')
submit_button.click()


daylinks = driver.find_elements_by_xpath('//a[@class="WSJTheme--day-link--19pByDpZ "][@href]')
time.sleep(2)

In [ ]:
raw_daylinks = []

# This will get all the articles per day
for elem in daylinks:
    raw_daylinks.append(elem.get_attribute('href'))
    
# Load that daylink page and extract all of the links to the individual articles

all_article_links = [] # will store all of the links to individual articles
article_links = [] # will store all unique links to individual articles after removing duplicates

for daylink in raw_daylinks:
    driver.get(daylink) 
    links = driver.find_elements_by_xpath(".//*[contains(@href, 'article')]")
    for link in links:
        all_article_links.append(link.get_attribute('href'))
        
        # Remove duplicate links
        for item in all_article_links:
            if item not in article_links:
                article_links.append(item)
                
                
# Making the right soup process
contents = []
# counter = 6

for link in article_links:
    driver.get(link)
    soup = BeautifulSoup(driver.page_source, 'lxml')
    contents.append(soup)

    time.sleep(1.5)
    

# Extracting the info needed from the Soup

article_dict = {}
for index, article in enumerate(contents):
    
    try: 
        
        # Get aricle headline
        article_headline = article.find('h1',{'class':'wsj-article-headline'})
        #print(article_headline)

        article_body = article.find('div', {'class': 'article-content'})
        #print(article_body.text)

        article_date = article.find('time', {'class': 'timestamp'})
        #print(article_date.text)

        # Append the extracted data to the articles dictionary
        article_dict['article_headline'] = article_headline.text.strip()
        article_dict['article_body'] = article_body.text.strip()
        article_dict['article_date'] = article_date.text.strip()
    
    except AttributeError:
        pass
    
    # Save data to CSV file . WARNING: Be careful to only run this once, as it keeps saving the data with every run
    file_exists = os.path.isfile('wsj_articles.csv')
    with open('wsj_articles.csv', 'a', encoding='utf-8', newline='') as csv_file:
        headers = ['headline', 'body', 'date']
        writer = csv.DictWriter(csv_file, delimiter=',', lineterminator='\n',fieldnames=headers)

        if not file_exists:
            writer.writeheader()

        writer.writerow({'headline': article_dict['article_headline'],
                         'body': article_dict['article_body'],
                         'date': article_dict['article_date']})


driver.close()